In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Stage 2: Building MVP: - 02 Chunk & Embeddings


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/workshops/rag-ops/2.2_mvp_chunk_embeddings.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fworkshops%2Frag-ops%2F2.2_mvp_chunk_embeddings.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/workshops/rag-ops/2.2_mvp_chunk_embeddings.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/workshops/rag-ops/2.2_mvp_chunk_embeddings.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## Overview

This notebook is the second in a series designed to guide you through building a Minimum Viable Product (MVP) for a Multimodal Retrieval Augmented Generation (RAG) system using the Vertex Gemini API.

Building upon the foundation laid in the first notebook, where we focused on extracting information from diverse data sources like PDFs, audio files, and video, this notebook dives deep into preparing that extracted textual data for efficient retrieval.

**Here's what you'll achieve:**

* **Master Text Chunking:** Learn how to effectively divide extracted text into manageable chunks optimized for embedding generation and subsequent retrieval.

* **Generate Powerful Embeddings:**  Understand the importance of embeddings in RAG systems and leverage Vertex AI Embeddings to transform text chunks into meaningful vector representations. We'll utilize the Batch mode feature of Vertex AI Embeddings with BigQuery backing to efficiently process thousands of text chunks in parallel.
* **Optimize for Scalability:**  Gain practical experience with asynchronous processing by using the Async Vertex Gemini API. This allows you to send multiple parallel requests for extracting text from multimodal data (audio and video), significantly speeding up your workflow.
* **Save and Store:**  Implement best practices for saving the intermediate files generated during the chunking and embedding process, ensuring data persistence and efficient retrieval in later stages of your RAG system development.

This notebook provides a crucial bridge between raw data extraction and the core retrieval functionality of your RAG system. By mastering text chunking and embedding generation, you'll be well-equipped to build a robust and scalable MVP.


## Getting Started

### Install Vertex AI SDK for Python


In [2]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

In [3]:
import sys

if "google.colab" in sys.modules:
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.


In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information, GCS Bucket and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
import os
import sys

from google.cloud import storage
import vertexai

PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"
BUCKET_NAME = "mlops-for-genai"

if PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

if not PROJECT_ID or PROJECT_ID == "[your-project-id]" or PROJECT_ID == "None":
    raise ValueError("Please set your PROJECT_ID")


vertexai.init(project=PROJECT_ID, location=LOCATION)

# Initialize cloud storage
storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(BUCKET_NAME)

In [4]:
# # Variables for data location. Do not change.

PRODUCTION_DATA = "multimodal-finanace-qa/data/unstructured/production/"
PICKLE_FILE_NAME = "data_extraction_dataframe.pkl"
EMBEDDING_INPUT_PATH = "multimodal-finanace-qa/data/embeddings"

### Import libraries


In [5]:
# Library

import pickle

from google.cloud import storage
import pandas as pd
from rich.markdown import Markdown as rich_Markdown
from vertexai.generative_models import (
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel

### Load the Gemini 1.5 models

To learn more about all [Gemini API models on Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models).

The Gemini model family has several model versions. You will start by using Gemini 1.5 Flash. Gemini 1.5 Flash is a more lightweight, fast, and cost-efficient model. This makes it a great option for prototyping.


In [6]:
MODEL_ID_FLASH = "gemini-1.5-flash-002"  # @param {type:"string"}
MODEL_ID_PRO = "gemini-1.5-pro-002"  # @param {type:"string"}


gemini_15_flash = GenerativeModel(MODEL_ID_FLASH)
gemini_15_pro = GenerativeModel(MODEL_ID_PRO)

In [7]:
# @title Gemini API Call Functions


def get_gemini_response(
    model,
    generation_config=None,
    safety_settings=None,
    uri_path=None,
    mime_type=None,
    prompt=None,
):
    if not generation_config:
        generation_config = {
            "max_output_tokens": 8192,
            "temperature": 1,
            "top_p": 0.95,
        }

    if not safety_settings:
        safety_settings = {
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        }

    uri = "gs://" + uri_path
    file = Part.from_uri(mime_type=mime_type, uri=uri)
    responses = model.generate_content(
        [file, prompt],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=True,
    )
    final_response = []
    for response in responses:
        try:
            final_response.append(response.text)
        except ValueError:
            # print("Something is blocked...")
            final_response.append("blocked")

    return "".join(final_response)


def get_load_dataframes_from_gcs():
    gcs_path = "multimodal-finanace-qa/data/structured/" + PICKLE_FILE_NAME
    # print("GCS PAth: ", gcs_path)
    blob = bucket.blob(gcs_path)

    # Download the pickle file from GCS
    blob.download_to_filename(f"{PICKLE_FILE_NAME}")

    # Load the pickle file into a list of dataframes
    with open(f"{PICKLE_FILE_NAME}", "rb") as f:
        dataframes = pickle.load(f)

    # Assign the dataframes to variables
    extracted_text, audio_metadata_flash, video_metadata_flash = dataframes

    return extracted_text, audio_metadata_flash, video_metadata_flash

![](https://storage.googleapis.com/mlops-for-genai/multimodal-finanace-qa/img/building_index_batch_processing_flow_animated.svg)

## Step 2.1: Data Chunking

![](https://storage.googleapis.com/mlops-for-genai/multimodal-finanace-qa/img/data_processing_step2.png)

In [8]:
# Get the data that has been extracted in the previous step: Data Processing.
# Make sure that you have ran the previous notebook: stage_2_mvp_data_processing.ipynb
extracted_text, audio_metadata_flash, video_metadata_flash = (
    get_load_dataframes_from_gcs()
)

<img src="https://storage.googleapis.com/gemini-lavi-asset/img/Step2-Chunking.png" width="500" />

In [9]:
# @title Text Chunking Helper Functions


def split_text_into_chunks(df, text_column, chunk_size):
    """Splits text into chunks of specified size, preserving other column values."""

    # Create a list of new dataframes, one for each chunk
    new_dfs = []
    for _, row in df.iterrows():
        text_chunks = [
            row[text_column][i : i + chunk_size]
            for i in range(0, len(row[text_column]), chunk_size)
        ]
        for chunk in text_chunks:
            new_row = row.copy()  # Copy all other columns
            new_row[text_column] = chunk
            new_dfs.append(pd.DataFrame([new_row]))

    return pd.concat(new_dfs, ignore_index=True)  # Combine into single dataframe

In [10]:
chunk_size = 500
extracted_text_chunk_df = split_text_into_chunks(extracted_text, "text", chunk_size)
video_metadata_chunk_df = split_text_into_chunks(
    video_metadata_flash, "video_description", chunk_size
)
audio_metadata_chunk_df = split_text_into_chunks(
    audio_metadata_flash, "audio_description", chunk_size
)

In [11]:
extracted_text_chunk_df.head()

In [12]:
audio_metadata_chunk_df.head(2)

In [13]:
video_metadata_chunk_df.head(2)

Original Page Text

In [ ]:
rich_Markdown(
    extracted_text[
        (extracted_text["page_number"] == 3)
        & (
            extracted_text["gcs_path"]
            == "gs://mlops-for-genai/multimodal-finanace-qa/data/unstructured/production/blogpost/Gemma on Google Kubernetes Engine deep dive _ Google Cloud Blog.pdf"
        )
    ]["text"].values[0]
)

Chunked Page text - Split into three chunks based on 500 characters.

In [ ]:
rich_Markdown(extracted_text_chunk_df.iloc[6]["text"])

In [ ]:
rich_Markdown(extracted_text_chunk_df.iloc[7]["text"])

In [ ]:
rich_Markdown(extracted_text_chunk_df.iloc[8]["text"])

## Step 2.2: Creating Embeddings

In [19]:
# @title Embedding Helper Functions


def get_text_embeddings(
    texts: list[str] = ["banana muffins? ", "banana bread? banana muffins?"],
    task: str = "RETRIEVAL_DOCUMENT",
    model_name: str = "textembedding-gecko@003",
) -> list[list[float]]:
    # print("doing...")
    """Embeds texts with a pre-trained, foundational model."""
    model = TextEmbeddingModel.from_pretrained(model_name)
    inputs = [TextEmbeddingInput(text, task) for text in texts]
    embeddings = model.get_embeddings(inputs)
    return [embedding.values for embedding in embeddings][0]


def backup_metadata_in_pickle(
    extracted_text, video_metadata, audio_metadata, index_db, output_path_with_name
):
    import pickle

    data_to_save = {
        "extracted_text": extracted_text,
        "video_metadata": video_metadata,
        "audio_metadata": audio_metadata,
        "index_db": index_db,
    }
    print("Backing up the metadata in: ", output_path_with_name + ".pkl")
    with open(f"{output_path_with_name}.pkl", "wb") as f:
        pickle.dump(data_to_save, f)


import uuid

uuid.uuid4()


def assign_unique_uuids(dataframes):
    """Assigns unique UUIDs to each row of multiple dataframes.

    Args:
        dataframes (list): A list of pandas DataFrames.

    Returns:
        list: A list of DataFrames with the 'uid' column added.
    """

    result_dataframes = []
    for df in dataframes:
        df["uid"] = df.apply(lambda row: str(uuid.uuid4().hex), axis=1)
        result_dataframes.append(df)

    return result_dataframes


import json


def create_jsonl_file(
    extracted_text_chunk_df,
    video_metadata_chunk_df,
    audio_metadata_chunk_df,
    bucket_object,
    jsonl_file_path,
):
    """
    Creates a JSONL file containing the combined text, video_description, and audio_description from the given dataframes.

    Args:
        extracted_text_chunk_df (pandas.DataFrame): The dataframe containing extracted text chunks.
        video_metadata_chunk_df (pandas.DataFrame): The dataframe containing video metadata.
        audio_metadata_chunk_df (pandas.DataFrame): The dataframe containing audio metadata.
    """

    json_data = []
    df_data = []

    for index, row in extracted_text_chunk_df.iterrows():
        json_data.append({"content": row["text"]})

        df_data.append([row["uid"], "text", row["text"]])

    for index, row in video_metadata_chunk_df.iterrows():
        json_data.append({"content": row["video_description"]})

        df_data.append([row["uid"], "video_description", row["video_description"]])

    for index, row in audio_metadata_chunk_df.iterrows():
        json_data.append({"content": row["audio_description"]})

        df_data.append([row["uid"], "audio_description", row["audio_description"]])

    # Convert the JSON data to a string
    jsonl_data = ""
    for item in json_data:
        jsonl_data += json.dumps(item) + "\n"

    # # Upload the JSONL data to GCS
    # blob = bucket.blob(jsonl_file_path+"/combined_data.jsonl")
    # blob.upload_from_string(jsonl_data)
    # print(f"File uploaded to GCS: {blob.public_url}")

    # gcs_path_jsonl_data = f"gs://{bucket.name}/{blob.name}"

    # return pd.DataFrame(json_data), pd.DataFrame(df_data, columns=['uid','type', 'content']), gcs_path_jsonl_data
    return pd.DataFrame(json_data), pd.DataFrame(
        df_data, columns=["uid", "type", "content"]
    )


def load_jsonl_from_gcs(bucket, file_path):
    """Loads a JSONL file from a GCS bucket and converts it into a DataFrame.

    Args:
        bucket_name (str): The name of the GCS bucket.
        file_path (str): The path to the JSONL file within the bucket.

    Returns:
        pandas.DataFrame: The DataFrame created from the JSONL data.
    """

    # storage_client = storage.Client()
    # bucket = storage_client.bucket(bucket_name)
    file_path_final = "/".join(file_path.split("//")[1].split("/")[1:])
    print(file_path_final)
    blob = bucket.blob(file_path_final)

    with blob.open("rb") as f:
        data = []
        for line in f:
            instance = json.loads(line)
            content = instance["instance"]["content"]
            predictions = instance["predictions"][0]["embeddings"]["values"]
            data.append({"content": content, "predictions": predictions})

    df = pd.DataFrame(data)
    return df

In [15]:
(extracted_text_chunk_df, video_metadata_chunk_df, audio_metadata_chunk_df) = (
    assign_unique_uuids(
        [extracted_text_chunk_df, video_metadata_chunk_df, audio_metadata_chunk_df],
    )
)

In [16]:
extracted_text_chunk_df.head(2)

In [17]:
video_metadata_chunk_df.head(2)

In [20]:
json_db_emb, index_db = create_jsonl_file(
    extracted_text_chunk_df,
    video_metadata_chunk_df,
    audio_metadata_chunk_df,
    bucket,
    EMBEDDING_INPUT_PATH,
)

In [21]:
index_db.head()

In [22]:
index_db.shape

You can skip this part and load the already processed embeddings.

In [ ]:
# gcs_path_jsonl_data

This code snippet grants necessary permissions to a service account to interact with Google Cloud Storage and Vertex AI. This is often a required step when setting up integrations between different Google Cloud services, such as using a Vertex AI model within BigQuery.

In [ ]:
# # # Refer here: https://cloud.google.com/bigquery/docs/generate-text-tutorial#grant-permissions

# # Grant the Storage Object Creator role to the service account associated with
# # the AI Platform service agent. This allows the service agent to write objects
# # to the specified Cloud Storage bucket.

# !gsutil iam ch \
#   "$(gcloud projects get-iam-policy $PROJECT_ID \
#   --flatten="bindings[].members" \
#   --filter="bindings.role:roles/aiplatform.serviceAgent" \
#   --format="value(bindings.members)"):roles/storage.objectCreator" \
#   gs://mlops-for-genai

# # # If you have multiple service accounts and getting an error: CommandException: Incorrect member type for binding serviceAccount:, use this:
# # !SERVICE_ACCOUNTS=$(gcloud projects get-iam-policy $PROJECT_ID    \
# #           --flatten="bindings[].members"                          \
# #           --filter="bindings.role:roles/aiplatform.serviceAgent"  \
# #           --format="value(bindings.members)")
# # !for SERVICE_ACCOUNT in $SERVICE_ACCOUNTS; do gsutil iam ch "$SERVICE_ACCOUNT:roles/storage.objectCreator" gs://mlops-for-genai; done

In [ ]:
# # # Generate Embeddings. It will take roughly 2-4 minutes

# from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel
# from vertexai.preview import language_models

# input_uri = (
#     gcs_path_jsonl_data
# )
# output_uri = "gs://mlops-for-genai/multimodal-finanace-qa/data/embeddings/combined_data_output/"

# textembedding_model = language_models.TextEmbeddingModel.from_pretrained(
#     "textembedding-gecko@003"
# )

# batch_prediction_job = textembedding_model.batch_predict(
#     dataset=[input_uri],
#     destination_uri_prefix=output_uri,
# )

You know your job is successful, if you see "JobState.JOB_STATE_SUCCEEDED" in the second last line. If you don't see that and get error, refer below.

if you receive the following error:

` "message: "Failed to run inference job. Query error: bqcx-xxxxxxxxxx-xxxx@gcp-sa-bigquery-condel.iam.gserviceaccount.com does not have the permission to access or use the endpoint. Please grant the Vertex AI user role to the service account following https://cloud.google.com/bigquery/docs/generate-text-tutorial#grant-permissions. If issue persists, contact bqml-feedback@google.com for help. at [4:1]""`

Copy the `"bqcx-xxxxxxxxxx-xxxx@gcp-sa-bigquery-condel.iam.gserviceaccount.com"` as is and run the following IAM permission command below. Example:

!gcloud projects add-iam-policy-binding $PROJECT_ID \
  --member="serviceAccount:bqcx-xxxxxxxxxx-xxxx@gcp-sa-bigquery-condel.iam.gserviceaccount.com" \
  --role="roles/aiplatform.user"

Make sure you get "version: 1" as the last line of the output. Then re-run the "# # Generate Embeddings " block, a cell above this one, before procedding further.

In [ ]:
# # Grant the AI Platform User role to a specific service account. This allows the
# # service account to use AI Platform resources, such as models.

# !gcloud projects add-iam-policy-binding $PROJECT_ID \
#   --member="serviceAccount:<replace with BQ service-account shown in error>" \
#   --role="roles/aiplatform.user"

In [23]:
# print(batch_prediction_job.display_name)
# print(batch_prediction_job.resource_name)
# print(batch_prediction_job.state)

In [24]:
# # Access the output information
# output_info = batch_prediction_job.output_info

# # The output_info is a list of dictionaries, each with a 'gcs_output_directory' key
# # In your case, you're expecting a single output file, so you can access it like this:
# output_dir = output_info.gcs_output_directory

# # Construct the full path to the JSONL file
# jsonl_file_path = f"{output_dir}/000000000000.jsonl"

# print(jsonl_file_path)

In [25]:
# If you want to run your own version of embeddings using Batch Vertex AI Embeddings, then you can run previous cells.

jsonl_file_path = "gs://mlops-for-genai/multimodal-finanace-qa/data/embeddings/combined_data_output/prediction-model-2024-10-29T19:19:00.826060Z/000000000000.jsonl"

In [26]:
%%time
embedding_df = load_jsonl_from_gcs(bucket, jsonl_file_path)

In [27]:
embedding_df.tail()

In [28]:
print("Size of embedding_df: ", embedding_df.shape)
print("Size of index_db: ", index_db.shape)

In [29]:
# Joining embedding_df with the index_df
index_db_final = index_db.merge(embedding_df, on="content", how="left")

In [30]:
index_db_final.head()

In [31]:
# to test if mapping is done right.
test_index = 5000
print(
    "*****original emb in embedding_db: *****\n",
    embedding_df.iloc[test_index]["predictions"][:5],
)
print(
    "\n*****emb in index_db****\n",
    index_db_final[
        index_db_final["content"] == embedding_df.iloc[test_index]["content"]
    ]["predictions"].values[0][:5],
)
print(
    "\n*****Original content in embedding_db *****",
    embedding_df.iloc[test_index]["content"],
)
print(
    "\n*****content in index_db*****",
    index_db_final[
        index_db_final["content"] == embedding_df.iloc[test_index]["content"]
    ]["content"].values[0],
)

In [32]:
index_db_final.value_counts("type")

In [33]:
index_db_final.columns = ["uid", "type", "content", "embeddings"]

### Save the intermediate Files

In [34]:
# # [Optional]

# import pickle

# pickle_file_name ="index_db.pkl"
# data_to_dump = [index_db_final, extracted_text_chunk_df,video_metadata_chunk_df,audio_metadata_chunk_df]

# gcs_location = f"gs://mlops-for-genai/multimodal-finanace-qa/data/embeddings/{pickle_file_name}"

# with open(f"{pickle_file_name}", "wb") as f:
#     pickle.dump(data_to_dump, f)


# # Upload the pickle file to GCS
# !gsutil cp {pickle_file_name} {gcs_location}